# COVID-19 in Poland
### Choropleth of confirmed COVID-19 cases in Poland

*Piotr Zawal*

This small project aims to map the number confirmed COVID-19 cases and related deaths (the data is taken from the website of Polish Ministry of Health) on the contour map of Poland.

**Important: to see rendered map, go to:**
https://nbviewer.jupyter.org/github/pzet/COVID_Poland/blob/master/covid_polska.ipynb

In [1]:
import pandas as pd
import folium
import json
import bs4
import requests

In [2]:
# download the geojson file
res_geojson = requests.get('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/poland.geojson')

# load geoJSON file
poland_geojson = json.load(open('poland.geojson', 'r', encoding='utf-8'))

In [3]:
# create a list with voivodships names
wojewodztwa = [nazwa.get('properties').get('name') for nazwa in poland_geojson.get('features')]
wojewodztwa = sorted(wojewodztwa)
print(wojewodztwa)

['Dolnośląskie', 'Kujawsko-Pomorskie', 'Lubelskie', 'Lubuskie', 'Mazowieckie', 'Małopolskie', 'Opolskie', 'Podkarpackie', 'Podlaskie', 'Pomorskie', 'Warmińsko-Mazurskie', 'Wielkopolskie', 'Zachodniopomorskie', 'Łódzkie', 'Śląskie', 'Świętokrzyskie']


In [4]:
# download the webpage containing the table with viral infections cases
res = requests.get('https://www.gov.pl/web/koronawirus/wykaz-zarazen-koronawirusem-sars-cov-2')

# parse the webpage
soup = bs4.BeautifulSoup(res.content, 'lxml')

# the data in the table has no specific class, but can be found using the text = True parameter
mz_html_text = soup.find_all(text = True)

table_to_parse = [element for element in mz_html_text if 'wielkopolskie' in element][0]

table_to_parse
# # convert to dictionary 
table_to_parse = json.loads(table_to_parse)
parsed_data = table_to_parse.get('parsedData')
# what we get is string representation of a list, which can be further converted to list with ast module
import ast
parsed_data = ast.literal_eval(parsed_data)

# convert to dataframe to add values to GeoJson
parsed_data_df = pd.DataFrame(parsed_data)
parsed_data_df

# data cleaning
## convert values in Liczba zgonów and Liczba to int type
parsed_data_df['Liczba'] = pd.to_numeric(parsed_data_df['Liczba'])
parsed_data_df['Liczba zgonów'] = pd.to_numeric(parsed_data_df['Liczba zgonów'])
## fill empty rows with 0s
parsed_data_df['Liczba zgonów'].fillna(0, inplace=True)
parsed_data_df['Liczba zgonów'] = parsed_data_df['Liczba zgonów'].astype(int)

## extract the total number of cases to show in the map legend
total_cases = parsed_data_df.iloc[0, 1]
## drop first row with summarized data for all country
parsed_data_df = parsed_data_df.iloc[1:]

## to assure compatibility with geojson file make sure that all the names are the same
### get names from geojson file
woj_names_list = sorted([x.get('properties').get('name') for x in poland_geojson.get('features')])
### sort voivodeship names alphabetically
parsed_data_df = parsed_data_df.sort_values('Województwo')
### assign properly formatted names
parsed_data_df['Województwo'] = woj_names_list
parsed_data_df


,Województwo,Liczba,Liczba zgonów,Id
1,Dolnośląskie,298,4,t02
2,Kujawsko-Pomorskie,76,0,t04
3,Lubelskie,132,4,t06
4,Lubuskie,47,0,t08
7,Mazowieckie,544,9,t14
6,Małopolskie,209,1,t12
8,Opolskie,73,0,t16
9,Podkarpackie,100,3,t18
10,Podlaskie,44,0,t20
11,Pomorskie,56,0,t22


In [5]:
for woj in poland_geojson.get('features'):
    woj_properties = woj.get('properties')
    # get voivodeship name from geojson file
    woj_name = woj.get('properties').get('name')
    #print(woj_properties)
    #print(woj_name)
    woj_properties.update({'cases' : str(parsed_data_df.loc[parsed_data_df['Województwo'] == woj_name, 'Liczba'].iloc[0])})
    woj_properties.update({'deaths' : str(parsed_data_df.loc[parsed_data_df['Województwo'] == woj_name, 'Liczba zgonów'].iloc[0])})
    #print(woj_properties)

In [6]:
m = folium.Map(location=[52, 19], tiles = 'cartodbpositron', zoom_start=6)

covid_map = folium.Choropleth(
                geo_data=poland_geojson,
                name='choropleth',
                data=parsed_data_df,
                columns=['Województwo', 'Liczba'],
                key_on='feature.properties.name',
                fill_color='YlOrRd',
                fill_opacity=0.7,
                line_opacity=0.2,
                legend_name='Potwierdzone przypadki COVID-19 (calkowita liczba : {})'.format(total_cases)
            ).add_to(m)


folium.LayerControl().add_to(m)

# add tooltip to covid_map.geojson file which is created under folium.Choropleth class
folium.GeoJsonTooltip(fields = ['name', 'cases', 'deaths'], 
                      aliases = ['Województwo: ', 'Przypadki: ', 'Liczba zgonów: ']
                     ).add_to(covid_map.geojson)

m